# make inverted index of domains -> users

### Import data

In [1]:
import os
from google.colab import drive


# tweets folder contains all tweets
drive.mount('/content/drive')
data = '/content/drive/My Drive/covid_project/graph_files'
print(data)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
/content/drive/My Drive/covid_project/graph_files


### Make dictionary < domain : accounts >

In [2]:
!pip install jsonlines

In [3]:
import glob
import jsonlines
from urllib.parse import urlparse

# list of 10 most cited domains, for them we use the full url of the page
most_cited = [ 'twitter.com', 'bit.ly', 'youtu.be', 'paper.li', 'ow.ly', 'www.instagram.com', 'www.pscp.tv', 'buff.ly', 'trib.al', 'www.youtube.com']

domains = {}
with jsonlines.open(data+"/sampled_tweets.jsonl") as infile:
    for line in infile:
        user = line['user']['name']

        # for each domain (or page) add to dict the user
        for domain in line['entities']['urls']:
          full_url = domain['expanded_url']
          dom = urlparse(full_url).netloc

          # check if use complete or not
          if dom in most_cited:
            # check if already in index
            if full_url in domains:
              # check if user already in domanin "posting list"
              if user not in domains[full_url]:
                  domains[full_url].append(user)
            else :
              domains[full_url] = [user]
          else:
            if dom in domains:
              if user not in domains[dom]:
                  domains[dom].append(user)
            else: 
              domains[dom] = [user]


In [4]:
print(f"Total domains/pages: {len(domains)}")
#print(domains)

Total domains/pages: 2647


In [5]:
import json
import jsonlines

file = data + '/inverted_domains.jsonl'

with jsonlines.open(file, mode='w') as outfile:
    for entry in domains:
      val = {'domain':entry, 'users':domains[entry]}
      outfile.write(val)